# Libraries

In [12]:
import pandas as pd
from getpass import getuser
from collections import defaultdict
from datetime import datetime, timedelta

# Load and inspect dataset

In [13]:
# Get the current user's name
user = getuser()

# Import the functions from the functions notebook
%run functions_uefa.ipynb


In [14]:


# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\in\wc_goals.xlsx'

# Load the Excel file into a DataFrame
df = pd.read_excel(data_path)

# Display the first few rows of the DataFrame to verify the import
display(df.head())

,stage,time,home_team,away_team,score,stadium_name,stadium_city,stadium_attendance,referee_name,referee_nationality,...,goal_minute,extra_time,goals_home,goals_away,own_goal,penalty,goal_minute_et,goal_et,short_date,long_date
0,Group A,20:45,China,Norway,4–0,Tianhe Stadium,Guangzhou,650008,Salvador Imperatore,Chile,...,22,0,4,0,0,0,0,0,1991-11-16,16 November 1991
1,Group A,20:45,China,Norway,4–0,Tianhe Stadium,Guangzhou,650008,Salvador Imperatore,Chile,...,45,0,4,0,0,0,0,0,1991-11-16,16 November 1991
2,Group A,20:45,China,Norway,4–0,Tianhe Stadium,Guangzhou,650008,Salvador Imperatore,Chile,...,50,0,4,0,0,0,0,0,1991-11-16,16 November 1991
3,Group A,20:45,China,Norway,4–0,Tianhe Stadium,Guangzhou,650008,Salvador Imperatore,Chile,...,75,0,4,0,0,0,0,0,1991-11-16,16 November 1991
4,Group A,19:45,Denmark,New Zealand,3–0,Tianhe Stadium,Guangzhou,140008,Omer Yengo,Congo,...,15,0,3,0,0,0,0,0,1991-11-17,17 November 1991


# Clean, transfrom, create variables

## time

In [15]:
# Step 1: Clean time variable by removing any letters (e.g., EEST, PST) if they exist
df['time_cleaned'] = df['time'].str.replace(r'[A-Za-z]+', '', regex=True).str.strip()

# Step 2: Split time where there are parentheses into local time and UTC offset
df['local_time'] = df['time_cleaned'].str.extract(r'(\d{2}:\d{2})')  # extract the part before parentheses
df['utc_time_offset'] = df['time_cleaned'].str.extract(r'\((.*?)\)')  # extract the part inside parentheses


## date

In [16]:
# Convert 'long_date' column to datetime format
df['long_date'] = pd.to_datetime(df['long_date'], format='%d %B %Y', errors='coerce')

# Extract the year from 'long_date' and create a new column 'year'
df['year'] = df['long_date'].dt.year


## gender

In [17]:
# Define the list of years for Men's FIFA World Cup editions
men_years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018, 2022, 2026]

# Create the 'men' variable: 1 if the year is in men_years_wc, 0 otherwise
df['men'] = df['year'].apply(lambda x: 1 if x in men_years else 0)


# Extract relevant columns

In [18]:
# Filter out the knockout stages
df = df[~df['stage'].isin(['Quarter-finals', 'Round of 16', 'Semi-finals', 'Final'])]

# Extract relevant columns for goal events and match results
goals_df = df[['men','year', 'stage', 'home_team', 'away_team', 'scorer_nationality', 'goal_minute', 'short_date','local_time', 'score']]

display(goals_df.head())


,men,year,stage,home_team,away_team,scorer_nationality,goal_minute,short_date,local_time,score
0,0,1991,Group A,China,Norway,China,22,1991-11-16,20:45,4–0
1,0,1991,Group A,China,Norway,China,45,1991-11-16,20:45,4–0
2,0,1991,Group A,China,Norway,China,50,1991-11-16,20:45,4–0
3,0,1991,Group A,China,Norway,China,75,1991-11-16,20:45,4–0
4,0,1991,Group A,Denmark,New Zealand,Denmark,15,1991-11-17,19:45,3–0


# Recreate Leauge Table after first two matchdays

In [19]:

# Assuming goals_df is your original dataset
agg_goals_before_last_day, goals_last_day_sorted = process_goals_data(df)


In [20]:
home_games, away_games = aggregate_home_away_points(agg_goals_before_last_day)

In [21]:
# Filter for year == 2021 and stage == 'Group F'
home_f_2021 = home_games[(home_games['year'] == 2022) & (home_games['stage'] == 'Group E')]
home_f_2021

,year,stage,home_team,men,goals_scored,goals_conceded,points_home,match_count_home
102,2022,Group E,Germany,1,1,2,0,1
103,2022,Group E,Japan,1,0,1,0,1
104,2022,Group E,Spain,1,8,1,4,2


In [22]:
# Filter for year == 2021 and stage == 'Group F'
away_f_2021 = away_games[(away_games['year'] == 2022) & (away_games['stage'] == 'Group E')]
away_f_2021

,year,stage,away_team,men,goals_scored,goals_conceded,points_away,match_count_away
99,2022,Group E,Costa Rica,1,1,7,3,2
100,2022,Group E,Germany,1,1,1,1,1
101,2022,Group E,Japan,1,2,1,3,1


## aggregate data after first two matches following 

### UEFA criteria (first h2h, then aggregate)

In [23]:
# Applying the fifa_before_last function to the mock data
all_games_before_last = uefa_before_last(home_games, away_games, agg_goals_before_last_day)

Tie in 1991, Stage: Group A, Men: 0, between China and Denmark (Points: 3.0)
Tie resolved by goals difference: China favored in 1991, Stage: Group A
Tie in 1991, Stage: Group B, Men: 0, between Brazil and Sweden (Points: 2.0)
Tie resolved by head-to-head: Sweden won in 1991, Stage: Group B
Tie in 1991, Stage: Group C, Men: 0, between Germany and Italy (Points: 4.0)
Tie resolved by goals difference: Germany favored in 1991, Stage: Group C
Tie in 1991, Stage: Group C, Men: 0, between Chinese Taipei and Nigeria (Points: 0.0)
Tie resolved by goals difference: Nigeria favored in 1991, Stage: Group C
Tie in 1995, Stage: Group A, Men: 0, between Brazil and Germany (Points: 3.0)
Tie resolved by goals scored: Germany favored in 1995, Stage: Group A
Tie in 1995, Stage: Group A, Men: 0, between Germany and Japan (Points: 3.0)
Tie resolved by head-to-head: Germany won in 1995, Stage: Group A
Tie in 1995, Stage: Group A, Men: 0, between Japan and Sweden (Points: 3.0)
Tie resolved by head-to-head: S

C:\Users\aldi\AppData\Local\Temp\ipykernel_19832\1092123632.py:106: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_games_before_last['standing'] = all_games_before_last.groupby(['year', 'stage', 'men']).apply(


# Recreate league table after last match day

### uefa criteria 

In [24]:
# Initialize an empty list to store the results for each pair
all_results = []

# Get unique pairs of year, stage, and men
unique_pairs = all_games_before_last[['year', 'stage', 'men']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    men = row['men']  # Define men based on the current row in unique_pairs
    
    # Apply the function to the current pair
    result = uefa_final_euro(year, stage, men, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_results.append(result)

# Concatenate all the results into a single DataFrame
changes_df_euro = pd.concat(all_results)

# Keep only the specified columns
changes_df_euro = changes_df_euro[['men', 'year', 'stage', 'team', '1st', '2nd', '3rd', '4th', 'changes']]



=== Initial Standings for Year 1991, Group A, Men: 0 Before Last Match Goals ===

       team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  before_last_game_points  before_last_game_standing
      China             4                   6                     2                      4                        3                          1
    Denmark             4                   5                     2                      3                        3                          2
     Norway             3                   4                     4                      0                        2                          3
New Zealand             1                   0                     7                     -7                        0                          4


Analyzing goal: 14 minute, Player team: Norway, Home: Norway, Away: Denmark

=== Updated Standings After This Goal ===

       team  total_points  total_goals_scored  total_goals_conceded  total_goal_

In [25]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\tb_wc_uefa.xlsx'
changes_df_euro.to_excel(file_path, index=False)


# group composition tracking

In [26]:
# Initialize an empty list to store the results for each pair
all_composition_changes = []

# Get unique pairs of year, stage, and men
unique_pairs = all_games_before_last[['year', 'stage', 'men']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    men = row['men']  # Get the value of 'men' for the current pair
    
    # Apply the function to the current pair
    composition_changes_df = track_composition_changes(year, stage, men, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_composition_changes.append(composition_changes_df)

# Concatenate all the results into a single DataFrame
final_composition_changes_df = pd.concat(all_composition_changes, ignore_index=True)

# Display the final DataFrame
display(final_composition_changes_df)




=== Initial Standings for Group A, 1991 (with 0-0 points added) ===
       team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  before_last_game_standing
      China             4                   6                     2                      4                          1
    Denmark             4                   5                     2                      3                          2
     Norway             3                   4                     4                      0                          3
New Zealand             1                   0                     7                     -7                          4



=== Initial Standings for Group B, 1991 (with 0-0 points added) ===
         team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  before_last_game_standing
United States             5                   8                     2                      6                          1
       Brazil             3   

,year,stage,men,change_num,goal_time,home_team,away_team,scorer_team,new_top_teams,1st,2nd,3rd
0,1991,Group A,0,0,initial,None,None,None,"[Norway, Denmark, China]",China,Denmark,Norway
1,1991,Group B,0,0,initial,None,None,None,"[Brazil, Sweden, United States]",United States,Sweden,Brazil
2,1991,Group C,0,0,initial,None,None,None,"[Italy, Germany, Chinese Taipei]",Germany,Italy,Nigeria
3,1995,Group A,0,0,initial,None,None,None,"[Germany, Brazil]",Germany,Sweden,Brazil
4,1995,Group A,0,1,5,Brazil,Germany,Germany,"[Germany, Sweden]",Germany,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
144,2023,Group G,0,2,66,Argentina,Sweden,Sweden,"[Sweden, South Africa]",Sweden,None,None
145,2023,Group G,0,3,67,South Africa,Italy,South Africa,"[Sweden, South Africa]",Sweden,None,None
146,2023,Group G,0,4,90,Argentina,Sweden,Sweden,"[Sweden, South Africa]",Sweden,None,None
147,2023,Group G,0,5,92,South Africa,Italy,South Africa,"[Sweden, South Africa]",Sweden,None,None


In [27]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\standings_wc_uefa.xlsx'
final_composition_changes_df.to_excel(file_path, index=False)
